In [9]:
import torch

N = 100
x = torch.linspace(0, 2*torch.pi, N)

dx1 = torch.mean(x[1:] - x[:-1])
dx2 = (2*torch.pi / (N-1))

print(dx1.item())
print(dx2)

print(dx1.item() - dx2)
print(torch.finfo(float).eps)

print((x[1] + 2*dx1 - x[3]).item())

0.06346651911735535
0.06346651825433926
8.63016089147628e-10
2.220446049250313e-16
0.0


In [18]:
x2 = torch.arange(0, 2*torch.pi+dx2, dx2)

In [40]:
torch.linspace(0, 10,50)

tensor([ 0.0000,  0.2041,  0.4082,  0.6122,  0.8163,  1.0204,  1.2245,  1.4286,
         1.6327,  1.8367,  2.0408,  2.2449,  2.4490,  2.6531,  2.8571,  3.0612,
         3.2653,  3.4694,  3.6735,  3.8776,  4.0816,  4.2857,  4.4898,  4.6939,
         4.8980,  5.1020,  5.3061,  5.5102,  5.7143,  5.9184,  6.1224,  6.3265,
         6.5306,  6.7347,  6.9388,  7.1429,  7.3469,  7.5510,  7.7551,  7.9592,
         8.1633,  8.3673,  8.5714,  8.7755,  8.9796,  9.1837,  9.3878,  9.5918,
         9.7959, 10.0000])

In [38]:
ns = torch.tensor([[1.0,0], [-1.0, 0], [0,1.0], [0,-1.0]])
vals = torch.tensor([1,2,3,4])
a = torch.tensor(10)
l = torch.tensor([1, 1.0]) / 2**0.5

#l[None, None].repeat(4,1,1)

print((torch.stack([ns, l.reshape(1,2).repeat(4,1)], dim=1)).shape)

a.nelement

print((torch.stack([vals, a[None].repeat(4)], dim=1)[...,None]).shape)
torch.stack([vals, a.reshape(1).repeat(4)], dim=1).reshape(4,2,1)

a.ne
#torch.concat([ns, l[None, None].repeat(4,1,1)], dim=1)

torch.Size([4, 2, 2])
torch.Size([4, 2, 1])


tensor([[[ 1],
         [10]],

        [[ 2],
         [10]],

        [[ 3],
         [10]],

        [[ 4],
         [10]]])

In [19]:
l.repeat

<function Tensor.repeat>

In [27]:
torch.nn.functional

<function torch.nn.functional.pad>

In [ ]:
def _project_forward(data: torch.Tensor, Xs: torch.Tensor, Ys: torch.Tensor, betas: torch.Tensor, us: torch.Tensor, R: float, DEVICE: torch.device):
    N_samples, H, W = data.shape
    Xs, Ys = Xs.reshape(-1), Ys.reshape(-1)
    Xmin, Xmax, Ymin, Ymax = Xs[0], Xs[-1], Ys[0], Ys[-1]
    dX, dY = torch.mean(Xs[1:]-Xs[:-1]), torch.mean(Ys[1:] - Ys[:-1])

    N_line_points = 2*(((Xmax - Xmin)**2 + (Ymax-Ymin)**2)**0.5 / dX).to(dtype=torch.int64)
    ratios = torch.arange(0, 1.0, 1 / N_line_points)[:, None]

    bounding_normals = torch.tensor([[1.0, 0.0], [-1.0, 0.0], [0.0, 1.0], [0.0, -1.0]], device=DEVICE, dtype=torch.float64)
    bounding_vals = torch.stack([Xmin, -Xmax, Ymin, -Ymax])

    betas, us = betas.reshape(-1, 1), us.reshape(1, -1)
    
    res = (betas * us * 0).repeat(N_samples, 1, 1)

    Nb, _ = betas.shape
    _, Nu = us.shape

    for i in range(Nb):
        for j in range(Nu):
            # Define line of Ray
            bi, uj = betas[i, 0], us[0, j]
            S, P = torch.stack([torch.cos(bi), torch.sin(bi)]) * R,  torch.stack([torch.sin(bi), -torch.cos(bi)])*uj
            line_dir = P - S
            line_dir /= torch.linalg.norm(line_dir)
            line_normal = torch.stack([line_dir[1], -line_dir[0]])
            tj = torch.sum(P*line_normal)

            # Find line segment in the region
            A = torch.stack([bounding_normals, line_normal.reshape(1, 2).repeat(4, 1)], dim=1)
            ts = torch.stack([bounding_vals, tj.reshape(1).repeat(4)], dim=1).reshape(4, 2, 1)
            intersections: torch.Tensor = torch.linalg.solve(A, ts).reshape(4, 2)
            # plot_hepler(data, torch.concat([S[None], P[None]]), dX, dY, 10)
            # draw_points_and_grid(Xmin, Xmax, Ymin, Ymax, torch.concat([S[None], P[None], intersections]), S[None] + line_dir[None]*torch.linspace(0,500, 1000)[:, None], line_normal[None]*torch.linspace(0,100,100)[:, None])
            on_inside = (torch.einsum("nk,ik->in", bounding_normals, intersections) - bounding_vals > -1e-5).sum(dim=-1) == 4
            if on_inside.sum() != 2:
                print(f"no intersections on border found for (i,j)={i},{j}")
                continue
            intersections = intersections[on_inside]
            # draw_points_and_grid(Xmin, Xmax, Ymin, Ymax, torch.concat([S[None], P[None], intersections]), S[None] + line_dir[None]*torch.linspace(0,500, 1000)[:, None], line_normal[None]*torch.linspace(0,100,100)[:, None])
            start, stop = intersections[0], intersections[1]
            if torch.sum((intersections[1]-intersections[0])*line_dir) < 0:
                start, stop = stop, start
            line_segment = stop - start
            line_points = start + ratios*line_segment.reshape(1, 2)
            dl = torch.linalg.norm(line_segment) / N_line_points

            #DEBUG
            # print("i,j", i, j)
            # print("dl", dl)
            # print("line_dir", line_dir)
            #DEBUG

            # Calculate linearly interpolated values of points along line
            inds_X, inds_Y = ((line_points[:, 0] - Xmin) / dX).to(torch.int64), ((line_points[:, 1]- Ymin) / dY).to(torch.int64)
            wxs, wys = (line_points[:, 0] - Xs[inds_X]) / dX, (line_points[:, 1] - Ys[inds_Y]) / dY

            vals = (data[:,H-1-inds_Y, inds_X]*(1-wxs)*(1-wys) +
                    data[:,H-1-(inds_Y+1), inds_X] * wxs * (1-wys) +
                    data[:,H-1-inds_Y, inds_X+1]*(1-wxs)*wys +
                    data[:,H-1-(inds_Y+1), inds_X+1] * wxs*wys
                )
            
            #DEBUG
            # if j > 60:
            #     print(wxs)
            #     print(wys)
            #     disp_img = torch.tensor(data[0].cpu()).to(torch.float64)
            #     disp_img[H-inds_Y, inds_X] = (vals[0] * 10 -5)

            #     plt.imshow(disp_img)
            #     plt.colorbar()
            #     plt.show()
            #     print("hello")
            # #DEBUG
            
            res[:, i, j] = torch.sum(vals, dim=-1)*dl

    return res